In [ ]:
import numpy as np
from FANS_Dashboard_utils import *
from postprocessing import compute_rank2tensor_measures
from h52xdmf import write_xdmf
from plotting import plot_subplots

In [ ]:
# Example usage
file_path = '../test/test_results.h5'

# Extract data from the file
hierarchy = identify_hierarchy(file_path)
summarize_hierarchy(hierarchy)

In [ ]:
# User specifies which microstructures, load cases, and quantities to load
microstructures_to_load = ['/sphere/32x32x32/ms']
load_cases_to_load = ['load0']
quantities_to_load = ['strain_average', 'stress_average', 'phase_stress_average_phase0', "stress"]
time_steps_to_load = []

# Extract the specified data, organized and sorted by time steps
data = extract_and_organize_data(file_path, hierarchy, 
                                 quantities_to_load, 
                                 microstructures_to_load, 
                                 load_cases_to_load, 
                                 time_steps_to_load)

In [ ]:
strain_average = data['/sphere/32x32x32/ms']['load0']['strain_average']
stress_average = data['/sphere/32x32x32/ms']['load0']['stress_average']
time_steps = data['/sphere/32x32x32/ms']['load0']['time_steps']

time_broadcasted = np.tile(time_steps[:, np.newaxis], (1, stress_average.shape[1]))

In [ ]:
# Specify additional measures to compute
measures_to_compute = ['von_mises', 'hydrostatic', 'deviatoric', 'principal', 
                       'max_shear', 'I_invariants', 'J_invariants', 'eigenvalues',
                       'eigenvectors', 'lode_angle']

# Compute the requested measures
results = compute_rank2tensor_measures(stress_average, measures_to_compute)
stress_average_deviatoric = results['deviatoric']

In [ ]:
# postprocessign and writing

quantities_to_process = ['stress_average', 'stress']
measures = ['deviatoric', 'von_mises', 'J_invariants']

# Process and write back
processed_data = postprocess_and_write_back(file_path, hierarchy, quantities_to_process, measures, 
                                microstructures_to_load, load_cases_to_load)

In [ ]:
# Plot stress average vs strain average for each component

plot_subplots(strain_average, stress_average, labels_x=['Strain']*6, labels_y=['Stress']*6, 
              subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'], 
              title='Stress average vs strain average', 
              nrows=2, ncols=3, linewidth=1, markersize=4, linecolor='blue', markercolor='blue', aspect_ratio=(1,1), fontsize=12)


In [ ]:
# Plot deviatoric stress average vs strain average for each component
plot_subplots(strain_average, stress_average_deviatoric, labels_x=['Strain']*6, labels_y=['Stress_dev']*6,
                subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'],
                title='Deviatoric stress average vs strain average',
                nrows=2, ncols=3, linewidth=1, markersize=4, linecolor='blue', markercolor='blue', aspect_ratio=(1,1), fontsize=12)

In [ ]:
# Plot deviatoric stress average vs time
plot_subplots(time_broadcasted, stress_average_deviatoric, labels_x=['Time']*6, labels_y=['Stress_dev']*6,
                subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'],
                title='Deviatoric stress average vs time',
                nrows=2, ncols=3, linewidth=1, markersize=4, linecolor='blue', markercolor='blue', aspect_ratio=(1,1), fontsize=12)


In [ ]:
plot_subplots(
    np.column_stack((strain_average[:, 0], strain_average[:, 0])), 
    np.column_stack((stress_average[:, 0], stress_average_deviatoric[:, 0])), 
    labels_x=['Strain_11']*2, labels_y=['Stress_11', 'Stress_dev_11'],
    subplot_titles=['Stress_11 vs Strain_11', 'Stress_dev_11 vs Strain_11'],
    nrows=1, ncols=2, linewidth=1, markersize=4, linecolor='blue', markercolor='blue', aspect_ratio=(1,1), fontsize=12)

In [ ]:
write_xdmf(file_path, time_series=True)